<a href="https://colab.research.google.com/github/jamiehadd/Math189AD-MathematicalDataScienceAndTopicModeling/blob/main/tutorials/Community_Detection_with_SymNMF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Math 189: Community Detection with SymNMF

In this notebook, we'll use our implementation of multiplicative updates for SymNMF to learn communities in a classical example for network analysis, Zachary's Karate Club!  This activity is adapted from a blog post from Toshi Takeuchi.

##**Activity**

We’re going to apply [an implementation](https://github.com/jamiehadd/Math189AD-MathematicalDataScienceAndTopicModeling/blob/main/sym_mult_ups.py) of multiplicative updates to train SymNMF on a standard dataset for community detection.

### Tasks

* Fill in any missing update code in our implementation of multiplicative updates.
* Run SymNMF on the Zachary Karate Club data and visualize the output.
* Run spectral clustering on the Zachary Karate Club data and visualize the output.
* There is lots of interesting network data out there for community detection!  Consider trying a different network (note that NetworkX has lots of cool examples built-in to their package).

##**Zachary's Karate Club**

The [Zachary's Karate Club](http://networkdata.ics.uci.edu/netdata/html/zacharyKarate.html) dataset contains a friendship network of 34 members of a karate club at a US university in the 1970s. No one knows exactly what happened, but [a dispute that erupted in this club eventually caused it to break up into two groups](https://books.google.com/books?id=atfCl2agdi8C&pg=PA64#v=onepage&q&f=false). Does the friendship network give enough information to predict how the group split?

In [ ]:
import scipy.io
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx

In [ ]:
url="https://raw.githubusercontent.com/jamiehadd/Math189AD-MathematicalDataScienceAndTopicModeling/main/sym_mult_ups.py"
!wget --no-cache --backups=1 {url}
from sym_mult_ups import sym_mult_ups

We'll visualize this network before we get to work determining the community structure.

In [ ]:
G = nx.karate_club_graph()

pos = nx.spring_layout(G, seed = 2)
nx.draw(G, pos, with_labels=True)
plt.title('Zacharys Karate Club')
plt.show()

Let's take a look at the adjacency matrix for this network!

In [ ]:
A = nx.adjacency_matrix(G).todense()

plt.imshow(A)

The eventual break-up produced two groups.  Let's visualize how the group broke up!

In [ ]:
G1nodes = np.array([1, 2, 3, 4, 5, 6, 7, 8, 9,11, 12, 13, 14, 17, 18, 20, 22])        # Node 1's group
G1nodes_python = G1nodes - 1
pos = nx.spring_layout(G, seed = 2)

# nodes
nx.draw_networkx_nodes(G, pos, node_color="tab:red")
nx.draw_networkx_nodes(G, pos, nodelist=G1nodes_python, node_color="tab:blue")

# edges
nx.draw_networkx_edges(G, pos, width=1.0, alpha=0.5)

plt.title('The club broke up in two groups')
plt.show()

##**SymNMF Community Detection**

Let's see if SymNMF can predict these two groups!

In [ ]:
M = 10000    #declare number of iterations
alpha = 1
newH,errs = sym_mult_ups(A,2,alpha,M)

In [ ]:
plt.imshow(newH)

In [ ]:
comm1nodes = np.where(newH[:,0] - newH[:,1] > 0)
print('Community 1: ', comm1nodes[0]+1)
comm2nodes = np.where(newH[:,0] - newH[:,1] < 0)
print('Community 2: ', comm2nodes[0]+1)

In [ ]:
pos = nx.spring_layout(G, seed = 2)

# nodes
nx.draw_networkx_nodes(G, pos, node_color="tab:red")
nx.draw_networkx_nodes(G, pos, nodelist=comm1nodes[0], node_color="tab:blue")

# edges
nx.draw_networkx_edges(G, pos, width=1.0, alpha=0.5)

plt.title('SymNMF predicted communities')
plt.show()

##**Spectral Clustering Community Detection**

Now, let's see if spectral clustering can detect these groups!  We use intuition from class that the second eigenvector contains sign information correlated with the groups.

In [ ]:
vals, vecs = np.linalg.eig(A)

In [ ]:
plt.imshow(np.real(vecs[:,1]))

In [ ]:
comm1nodes = np.where(vecs[:,1] > 0)
print('Community 1: ', comm1nodes[0]+1)
comm2nodes = np.where(vecs[:,1] < 0)
print('Community 2: ', comm2nodes[0]+1)

In [ ]:
pos = nx.spring_layout(G, seed = 2)

# nodes
nx.draw_networkx_nodes(G, pos, node_color="tab:red")
nx.draw_networkx_nodes(G, pos, nodelist=comm1nodes[0], node_color="tab:blue")

# edges
nx.draw_networkx_edges(G, pos, width=1.0, alpha=0.5)

plt.title('Spectral Clustering predicted communities')
plt.show()